In [1]:
include("../../../julia/FNC_init.jl")

  Activating project at `~/Documents/GitHub/fnc`


[**Demo %s**](#demo-newtonsys-converge)

A system of nonlinear equations is defined by its residual and Jacobian.
```{tip}
:class: dropdown
Be careful when coding a Jacobian all in one statement. Spaces separate columns, so `x[3]-1` is not the same as `x[3] - 1`.
```

In [2]:
function func(x)
    [exp(x[2] - x[1]) - 2,
        x[1] * x[2] + x[3],
        x[2] * x[3] + x[1]^2 - x[2]
    ]
end;

function jac(x)
    [
        -exp(x[2] - x[1]) exp(x[2] - x[1]) 0
        x[2] x[1] 1
        2*x[1] x[3]-1 x[2]
    ]
end;

We will use a `BigFloat` starting value, and commensurately small stopping tolerances, in order to get a sequence long enough to measure convergence.

In [3]:
x₁ = BigFloat.([0, 0, 0])
ϵ = eps(BigFloat)
x = FNC.newtonsys(func, jac, x₁, xtol=ϵ, ftol=ϵ);

Let's compute the residual of the last result in order to check the quality.

In [4]:
r = x[end]
@show residual = norm(func(r));

residual = norm(func(r)) = 0.0

We take the sequence of norms of errors, applying the log so that we can look at the exponents.

In [5]:
logerr = [Float64(log(norm(r - x[k]))) for k in 1:length(x)-1]
ratios = [NaN; [logerr[i+1] / logerr[i] for i in 1:length(logerr)-1]]
@pt :header=["iteration", "log error", "ratio"] [eachindex(logerr) logerr ratios]

iteration,log error,ratio
1.0,-0.642465,NaN
2.0,-0.509988,0.793799
3.0,-1.88491,3.69598
4.0,-4.58533,2.43266
5.0,-10.5971,2.31109
6.0,-22.5988,2.13254
7.0,-45.8704,2.02977
8.0,-93.3004,2.034


The ratio is neatly converging toward 2, which is expected for quadratic convergence.